### outil général pour checker le sfm data, les matches, aussi les photos non placées

+ upgrade plot_camera_poses avec orientation des cameras

In [1]:
import json
import pandas as pd
import numpy as np
import os
from glob import glob

SFM_MOUNT = '/media/gargantua/1000-plane/0000-sfm/'

In [2]:
# input : svg file ("Geometric" or "Putative" Adjacency Matrix) for a given model
# output : list of matches
def read_adj_mat(model, matrix): 
    matches = []
    svg_file = SFM_MOUNT + "/" + model[0:4] + "/" + model[4:8] + "/" + model + "/systems/omvg/matches/" + matrix + "AdjacencyMatrix.svg"
    for line in open(svg_file, 'r').read().split('\n'):
        if "<rect" in line:
            x = ( np.float(line.split('"')[1]) / 5 )
            y = ( np.float(line.split('"')[3]) / 5 )
            matches.append(sorted([int(x),int(y)]))
    return matches

# input : list of matches
# output : matrix to display/save
def disp_matches(matches):
    m = np.zeros((max(max(matches)) + 1, max(max(matches)) + 1))

    for pair in matches:
        i = pair[0]
        j = pair[1]
        m[i,j] = 1
    return m



In [13]:
#def sfm_info(path): # takes either a model or a full path to the json file as input
sfm_data_path = SFM_MOUNT + "/" + model[0:4] + "/" + model[4:8] + "/" + model + "/systems/omvg/sfm_data.json"
if not os.path.exists(sfm_data_path):
    sfm_data_path = model 

with open(sfm_data_path) as sfm_data:
    data = json.load(sfm_data)
    views = data['views']
    extrinsics = data['extrinsics']
    intrinsics = data['intrinsics']
    structure = data['structure']
    control_points = data['control_points'] # sert pas à grand chose pour l'instant


In [ ]:
views_keys = []
files = []

for view in views:
    views_keys.append(view['key'])
    files.append(view['value']['ptr_wrapper']['data']['filename'].replace('.tif',''))

key_to_file_dict = dict(zip(views_keys,files))

extrinsics_keys = []
camera_poses = []
for extrinsic in extrinsics:
    extrinsics_keys.append(extrinsic['key'])
    camera_poses.append(extrinsic['value']['center'])

key_to_pose_dict = dict(zip(extrinsics_keys,camera_poses))
pd.DataFrame([key_to_file_dict,key_to_pose_dict]).transpose().rename(columns = {0:'filename',1:'coor'})
